# 问答机器人-从问题中提取相关实体和关系

[同济子豪兄](https://space.bilibili.com/1900783) 2022-6-25

## 导入工具包

In [1]:
import os
import ahocorasick
from tqdm import tqdm

## 进入主目录

In [2]:
os.chdir(r'C:\Users\bistu\Downloads\QASystemOnMedicalKG-master')

In [3]:
cur_dir = os.getcwd()

## 载入所有特征词

In [4]:
#　特征词txt文件路径
disease_path = os.path.join(cur_dir, 'dict/disease.txt')
department_path = os.path.join(cur_dir, 'dict/department.txt')
check_path = os.path.join(cur_dir, 'dict/check.txt')
drug_path = os.path.join(cur_dir, 'dict/drug.txt')
food_path = os.path.join(cur_dir, 'dict/food.txt')
producer_path = os.path.join(cur_dir, 'dict/producer.txt')
symptom_path = os.path.join(cur_dir, 'dict/symptom.txt')
deny_path = os.path.join(cur_dir, 'dict/deny.txt')

In [5]:
# 加载特征词
disease_wds = [i.strip() for i in open(disease_path,encoding="utf-8") if i.strip()]
department_wds = [i.strip() for i in open(department_path,encoding="utf-8") if i.strip()]
check_wds = [i.strip() for i in open(check_path,encoding="utf-8") if i.strip()]
drug_wds = [i.strip() for i in open(drug_path,encoding="utf-8") if i.strip()]
food_wds = [i.strip() for i in open(food_path,encoding="utf-8") if i.strip()]
producer_wds = [i.strip() for i in open(producer_path,encoding="utf-8") if i.strip()]
symptom_wds = [i.strip() for i in open(symptom_path,encoding="utf-8") if i.strip()]
region_words = set(disease_wds + department_wds + check_wds + drug_wds + food_wds + producer_wds + symptom_wds)
deny_words = [i.strip() for i in open(deny_path,encoding="utf-8") if i.strip()]

## 所有实体对应的类型

In [6]:
wdtype_dict = dict()
for word in tqdm(region_words):
    wdtype_dict[word] = []
    if word in disease_wds:
        wdtype_dict[word].append('disease')
    if word in department_wds:
        wdtype_dict[word].append('department')
    if word in check_wds:
        wdtype_dict[word].append('check')
    if word in drug_wds:
        wdtype_dict[word].append('drug')
    if word in food_wds:
        wdtype_dict[word].append('food')
    if word in symptom_wds:
        wdtype_dict[word].append('symptom')
    if word in producer_wds:
        wdtype_dict[word].append('producer')

100%|██████████████████████████████████████████████████████████████████████████| 43430/43430 [00:37<00:00, 1149.19it/s]


In [7]:
wdtype_dict

{'拜耳先灵葆雅吸入用伊洛前列': ['producer'],
 '解郁安神颗粒': ['drug'],
 '栗子（熟）': ['food'],
 '潜江制药注射用尿激酶': ['producer'],
 '低磷血症': ['symptom'],
 '新亚闵行甲钴胺片': ['producer'],
 '佐力乙酰螺旋霉素片': ['producer'],
 '尿中持续性排钠': ['symptom'],
 '果脯土豆饼': ['food'],
 '五0五千柏鼻炎片': ['producer'],
 '广东台城制药去痛片': ['producer'],
 '辅仁注射用硫酸长春新碱': ['producer'],
 'C5功能不全综合征': ['disease'],
 '云南梁河民族双黄消炎片': ['producer'],
 '膀胱注水试验': ['check'],
 '华西制药复方斑蝥胶囊': ['producer'],
 '楚雄老拨云堂拨云眼膏': ['producer'],
 '山东华鲁复方甘露醇注射液': ['producer'],
 '氯喹那多/普罗雌烯阴道片': ['drug'],
 '杜仲党参煲乳鸽': ['food'],
 '江苏奥赛康药业注射用醋酸奥': ['producer'],
 '胸廓畸形': ['disease', 'symptom'],
 '胆道肿瘤': ['disease'],
 '大便频繁': ['symptom'],
 '壮腰健肾丸': ['drug'],
 '千金协力药业雷公藤多甙片': ['producer'],
 '委内瑞拉马脑炎': ['disease'],
 '甲状腺肿瘤': ['disease'],
 '前列舒乐胶囊': ['drug'],
 '长春银诺克清胰利胆颗粒': ['producer'],
 '老年人甲减性心脏病': ['disease'],
 '迪冉郸城依托红霉素颗粒': ['producer'],
 '开角型青光眼': ['disease'],
 '芒果凤尾虾': ['food'],
 '湖北御金丹明目地黄丸': ['producer'],
 '地纳制药乳酸菌素片': ['producer'],
 '尿液颜色': ['check'],
 '圣鲁制药酒石酸美托洛尔注射': ['producer'],
 '河南润

## 构造AC自动机，加速实体提取

当知识图谱规模过大时，使用AC自动机算法，从问题中提取知识图谱中的实体，而不是用原生python的字符串方法。

扩展阅读：https://zhuanlan.zhihu.com/p/158767004

In [8]:
'''构造AC自动机，加速过滤'''
def build_actree(wordlist):
    actree = ahocorasick.Automaton()
    for index, word in enumerate(wordlist):
        actree.add_word(word, (index, word))
    actree.make_automaton()
    return actree

In [9]:
# 构造AC自动机
region_tree = build_actree(list(region_words))

In [10]:
region_tree

In [11]:
'血常规' in region_tree

True

In [12]:
region_tree.get('血常规')

(7169, '血常规')

In [13]:
'学习' in region_tree

False

In [14]:
region_tree.get('学习')

KeyError: 

## 任务一：提取问题相关实体及其类别

In [15]:
question = '肺气肿和百日咳要做血常规吗'

In [16]:
question_entity = []
for each in region_tree.iter(question):
    print(each)
    entity = each[1][1]
    question_entity.append(entity)
print(question_entity)

(2, (32112, '肺气肿'))
(6, (7690, '百日咳'))
(11, (7169, '血常规'))
['肺气肿', '百日咳', '血常规']


In [17]:
# stop_wds = []
# # 排除字符串子串
# for wd1 in region_wds:
#     for wd2 in region_wds:
#         if wd1 in wd2 and wd1 != wd2:
#             stop_wds.append(wd1)
# print(stop_wds)
# final_wds = [each for each in region_wds if each not in stop_wds]
# final_wds

In [18]:
question_entity_dict = {each:wdtype_dict[each] for each in question_entity}

In [19]:
question_entity_dict

{'肺气肿': ['disease'], '百日咳': ['disease'], '血常规': ['check']}

In [20]:
output = {}
output['args'] = question_entity_dict

## 任务一完成

## 任务二：提取问题相关的待查询关系

### 问题中涉及的实体类别

In [21]:
types = []
for each in question_entity_dict.values():
    types.extend(each)

In [22]:
types

['disease', 'disease', 'check']

### 不同提问意图的疑问词

In [23]:
symptom_qwds = ['症状', '表征', '现象', '症候', '表现']
cause_qwds = ['原因','成因', '为什么', '怎么会', '怎样才', '咋样才', '怎样会', '如何会', '为啥', '为何', '如何才会', '怎么才会', '会导致', '会造成']
acompany_qwds = ['并发症', '并发', '一起发生', '一并发生', '一起出现', '一并出现', '一同发生', '一同出现', '伴随发生', '伴随', '共现']
food_qwds = ['饮食', '饮用', '吃', '食', '伙食', '膳食', '喝', '菜' ,'忌口', '补品', '保健品', '食谱', '菜谱', '食用', '食物','补品']
drug_qwds = ['药', '药品', '用药', '胶囊', '口服液', '炎片']
prevent_qwds = ['预防', '防范', '抵制', '抵御', '防止','躲避','逃避','避开','免得','逃开','避开','避掉','躲开','躲掉','绕开',
                     '怎样才能不', '怎么才能不', '咋样才能不','咋才能不', '如何才能不',
                     '怎样才不', '怎么才不', '咋样才不','咋才不', '如何才不',
                     '怎样才可以不', '怎么才可以不', '咋样才可以不', '咋才可以不', '如何可以不',
                     '怎样才可不', '怎么才可不', '咋样才可不', '咋才可不', '如何可不']
lasttime_qwds = ['周期', '多久', '多长时间', '多少时间', '几天', '几年', '多少天', '多少小时', '几个小时', '多少年']
cureway_qwds = ['怎么治疗', '如何医治', '怎么医治', '怎么治', '怎么医', '如何治', '医治方式', '疗法', '咋治', '怎么办', '咋办', '咋治']
cureprob_qwds = ['多大概率能治好', '多大几率能治好', '治好希望大么', '几率', '几成', '比例', '可能性', '能治', '可治', '可以治', '可以医']
easyget_qwds = ['易感人群', '容易感染', '易发人群', '什么人', '哪些人', '感染', '染上', '得上']
check_qwds = ['检查', '检查项目', '查出', '检查', '测出', '试出']
belong_qwds = ['属于什么科', '属于', '什么科', '科室']
cure_qwds = ['治疗什么', '治啥', '治疗啥', '医治啥', '治愈啥', '主治啥', '主治什么', '有什么用', '有何用', '用处', '用途', '有什么好处', '有什么益处', '有何益处', '用来', '用来做啥', '用来作甚', '需要', '要']

In [24]:
def check_words(words, question):
    # 疑问词是否出现在提问中
    for word in words:
        if word in question:
            # print(word + ' 出现在提问中')
            return True
    return False

In [25]:
# question_type = 'others'
question_types = []

# 疾病-症状
if check_words(symptom_qwds, question) and ('disease' in types):
    question_type = 'disease_symptom'
    question_types.append(question_type)
    
# 症状-疾病
if check_words(symptom_qwds, question) and ('symptom' in types):
    question_type = 'symptom_disease'
    question_types.append(question_type)
    
# 疾病-病因
if check_words(cause_qwds, question) and ('disease' in types):
    question_type = 'disease_cause'
    question_types.append(question_type)

# 疾病-并发症
if check_words(acompany_qwds, question) and ('disease' in types):
    question_type = 'disease_acompany'
    question_types.append(question_type)

# 疾病-可以吃/不可以吃
if check_words(food_qwds, question) and 'disease' in types:
    deny_status = check_words(deny_words, question)
    if deny_status:
        question_type = 'disease_not_food'
    else:
        question_type = 'disease_do_food'
    question_types.append(question_type)
    
# 可以吃/不可以吃-疾病
if check_words(food_qwds+cure_qwds, question) and 'food' in types:
    deny_status = check_words(deny_words, question)
    if deny_status:
        question_type = 'food_not_disease'
    else:
        question_type = 'food_do_disease'
    question_types.append(question_type)
    
# 疾病-药物
if check_words(drug_qwds, question) and 'disease' in types:
    question_type = 'disease_drug'
    question_types.append(question_type)
    
# 药物-疾病
if check_words(cure_qwds, question) and 'drug' in types:
    question_type = 'drug_disease'
    question_types.append(question_type)
    
# 疾病-检查
if check_words(check_qwds, question) and 'disease' in types:
    question_type = 'disease_check'
    question_types.append(question_type)
    
# 检查-疾病
if check_words(check_qwds+cure_qwds, question) and 'check' in types:
    question_type = 'check_disease'
    question_types.append(question_type)
    
#　疾病-预防措施
if check_words(prevent_qwds, question) and 'disease' in types:
    question_type = 'disease_prevent'
    question_types.append(question_type)
    
# 疾病-疗程
if check_words(lasttime_qwds, question) and 'disease' in types:
    question_type = 'disease_lasttime'
    question_types.append(question_type)

# 疾病-疗法
if check_words(cureway_qwds, question) and 'disease' in types:
    question_type = 'disease_cureway'
    question_types.append(question_type)

# 疾病-治愈率
if check_words(cureprob_qwds, question) and 'disease' in types:
    question_type = 'disease_cureprob'
    question_types.append(question_type)

# 疾病-易感人群
if check_words(easyget_qwds, question) and 'disease' in types :
    question_type = 'disease_easyget'
    question_types.append(question_type)

In [26]:
question_types

['check_disease']

In [27]:
# 若没有查到相关的外部查询信息，那么则将该疾病的描述信息返回
if question_types == [] and 'disease' in types:
    question_types = ['disease_desc']

# 若没有查到相关的外部查询信息，那么则将该疾病的描述信息返回
if question_types == [] and 'symptom' in types:
    question_types = ['symptom_disease']

# 将多个分类结果进行合并处理，组装成一个字典
output['question_types'] = question_types

In [28]:
output

{'args': {'肺气肿': ['disease'], '百日咳': ['disease'], '血常规': ['check']},
 'question_types': ['check_disease']}